# Preference Tuning for Summarization using Synthetic Data

**⏱️ Time to complete**: 10 hours+

Alignment of LLMs has traditionally been broken down into two post-training stages: Supervised fine-tuning (SFT) followed by preference tuning (aka RLHF). SFT requires high quality data collection where each data sample illustrates behavior which we would like the LLM to imitate exactly. While for some tasks like SQL generation and math reasoning, it is feasible to collect the ground truth data, this approach does not always scale easily to align for subjective use cases (ex. chat, summarization, etc.). 

On the other hand, preference tuning only requires information about whether a given response is preferred to another response. Each data sample consists of a chosen and rejected completion for a given prompt, such that the chosen completion is preferred over the rejected completion. Preference tuning is thus a powerful tool that can optimize LLMs towards complex preferences that cannot be easily captured through supervised fine-tuning. However, manually annotating preferences between model outputs using human raters can be extremely time-consuming and expensive. Instead, synthetic preference data can be generated by scoring responses with large foundation models, allowing for much cheaper and scalable data collection!

Here we'll go through an end-to-end example for preference tuning of an open-source language model with synthetic data, covering scalable methodologies for data preprocessing, fine-tuning and evaluation, using Ray. We will focus on the task of summarization for the [CNN/DailyMail](https://huggingface.co/datasets/abisee/cnn_dailymail) dataset. 


Notebook guide:
- 🔄 REPLACE indicates to replace with your unique values
- 💡 INSIGHT indicates infrastructure insight

# Table of Contents
1. [Data Preprocessing](#step-1-data-preprocessing): In this section we cover how we can prepare preference data for the summarization task using an LLM-as-a-judge. 
    1. [Generate Multiple Choice Questions From Articles](#part-a-generate-multiple-choice-questions-from-articles)
    2. [Generate Summaries and Scores](#part-b-generate-summaries--scores)
    3. [Generate Preference Tuning Data](#part-c-generate-preference-tuning-data)
2. [DPO Finetuning](#step-2-fine-tuning): This section will cover how you can fine-tune an open source model on the preference data on the Anyscale platform.
3. [Evaluation](#step-3-evaluation): The section will lay down a blue-print for evaluation and compare performance to that of closed source models like OpenAI's GPT-4.

**NOTE**: Running the jobs in this notebook requires a HuggingFace token that can access [Llama-3.1-70B-Instruct](https://huggingface.co/meta-llama/Meta-Llama-3.1-70B-Instruct) and [Mistral-7B-Instruct-v0.1](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1). For GPT-4o evaluation, you'd also need a valid `OPENAI_API_KEY`.  Make sure to provide `HF_TOKEN` and `OPENAI_API_KEY` by defining it under dependencies in your cluster setup.

<p align="center">
  <img src="./assets/env_var.png?" alt="Environment variable" width=800>
</p>

First, let's make the necessary imports

In [1]:
import os
import pprint

import ray.data
import datasets

from src.utils.models import DataSchema
from src.utils.common import print_wrapped

os.environ["PYTHONPATH"] = f"{os.environ.get('PYTHONPATH', '')}:src"

# Step 1: Synthetic Data Generation

First, let's inspect the training dataset and look at an example. 

In [ ]:
hf_ds = datasets.load_dataset("abisee/cnn_dailymail", "3.0.0", split="train")

raw_example = hf_ds[0]

In [3]:
pprint.pprint(raw_example, width=80)

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe '
            'gains access to a reported £20 million ($41.1 million) fortune as '
            "he turns 18 on Monday, but he insists the money won't cast a "
            'spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter '
            'and the Order of the Phoenix" To the disappointment of gossip '
            'columnists around the world, the young actor says he has no plans '
            'to fritter his cash away on fast cars, drink and celebrity '
            'parties. "I don\'t plan to be one of those people who, as soon as '
            'they turn 18, suddenly buy themselves a massive sports car '
            'collection or something similar," he told an Australian '
            'interviewer earlier this month. "I don\'t think I\'ll be '
            'particularly extravagant. "The things I like buying are things '
            'that cost about 10 pounds -- books and CDs and DVDs." At 18, '
 

Consider the example article above. Our goal is to train a model to summarize it accurately, with a preference for short summaries, making sure we also preserve important details. In this guide, we will employ a _synthetic_ summary scoring method using another LLM as a judge. We score the correctness of a summary using the following metrics:

**Summary Scoring Metrics**
1. Multiple choice Q&A accuracy:
    - Given the original text, we use an LLM judge to generate 5 multiple choice questions about the text.
    - We then ask the LLM judge to answer the questions using only the summary, and record the number of questions correctly answered.
2. Word count: We simply count the number of words in the summary.

This allows us to construct a simple preference function between two summaries:

**Preference Function**
1. If both summary responses attain more than 3/5 multiple choice questions correct, we will prefer the shorter response. We do not care about Q&A accuracy beyond 3 correct answers, since the summary should not contain all information from the text.
2. Otherwise, we select the response that leads to more correctly answered multiple choice questions.

We consider a subset of 21,000 articles in this example. To generate the preference pairs, we will generate 10 summaries from each article using the model we wish to fine-tune. Then, we will randomly sample pairs of summaries and use our preference function to annotate the preference between them.

For this example, we will use `Mistral-7B-Instruct-v0.1` as the base model to fine-tune and `Llama-3.1-70B-Instruct` as a judge. Note that mistral-instruct is already instruction tuned, so that given a prompt to do summarization it might do a good job, but it may not be aligned with how we want the summarization to look like. We can use preference data to further align the instruct variant towards our specific needs.

We've provided a helpful visualization here:

<p align="center">
  <img src="./assets/preference_function.png?" alt="Preference function" width=800>
</p>

Combining all this together, our data pre-processing pipeline is going to look as follows: 

![preprocessing](./assets/preprocessing.png?1)

💡 INSIGHT: 
Our synthetic preference data collection looks pretty involved at first glance. The key ideas in plain English are as follows:
- We use a combination of Q&A scoring + word length to indicate like/dislike (our preference function) given a pair of summaries.
- Our ultimate goal is to generate (chosen, rejected) pairs to train our reference model and evaluate it based on this criteria.
- We use another LLM (judge model) to generate said questions for each article. This model is also used in our scoring system. (To see how many questions can be answered from a summary)
- To generate training data, we first sample candidate summaries from the reference model for each article. We then obtain scores for each summary from the judge. Using the scores, we select pairs of summaries and mark our like/dislike to form (chosen, rejected) pairs for the actual training. 


### Part (a): Generate Multiple Choice Questions from Articles

First, we will generate the multiple choice questions and answers for each article using `Llama-3.1-8B-Instruct` (or `70B` if A100/H100s are available). Leveraging vLLM and Ray, we can very easily scale this generation process across multiple GPUs.

>  **_NOTE:_**  We provide two sets of configs: One with an 8B parameter model as the judge, and another with the 70B model. Using the 8B model is recommended, since we make use of highly available A10Gs. For good performance, and to replicate the results in our blog, you should use the 70B judge model which uses A100s (but these are harder to obtain on-demand)

The following command will run the [src/scripts/generate_questions.py](./src/scripts/generate_questions.py) script, which generates the questions and answers and saves them in `.parquet` files.


💡 INSIGHT:  
We are running this script as an anyscale job. The resources required by each step are requested at runtime and provisioned by Anyscale's autoscaler based on availability and quotas. You are free to change the [qa_generation](./configs/qa_generation) config in any way. The important parameters regarding resources are `accelerator_type`, `num_gpus_per_instance`, and `concurrency`. This script will generate 5 multiple choice question and answer pairs per article for 21k examples. According to the [llama_8b](./configs/qa_generation/llama_8b.yaml) config we are requesting 3 replicas of 4xA10G machines processing a batch-size of 128 examples each which saturates the GPUs all the way through.

This step will take ~40 min for 8B running on 12 A10s (~ $10) and ~75 mins for 70B running on 8 A100s (~ $28). 

```bash
anyscale job submit -f configs/jobs/8b_judge/generate_questions_job.yaml
# Optional: use the 70b model for better performance (runs on A100s)
# anyscale job submit -f configs/jobs/70b_judge/generate_questions_job.yaml
```

> **NOTE**: We recommend that you execute all the commands in this notebook in a terminal. Make sure you `cd` into the directory of this notebook (and the `src` files) before executing the commands. 

> **NOTE**: The default configurations provided are not tuned for maximum throughput. Feel free to modify the scaling configs (i.e concurrency, num gpus per instance) etc as needed (and as permitted by availability).

At the end of the job, you should see the remote path to the folder with Q&A in the logs.


<p align="center">
  <img src="./assets/question_generation_done.png?" alt="Evaluation" width=800>
</p>

 Make sure to make note to use it for the next steps! 

 🔄 REPLACE the resulting S3 URI here. If you want to skip the prior step, you can continue with the prepared example data below.

In [ ]:
# Replace this with the link to the output folder from the previous job
qa_folder = "s3://air-example-data/preference-tuning-summarization-example/qa_generation/qa_annotations_full_train/"
qa_ds = ray.data.read_parquet(qa_folder)
# The dataset is small, we can materalize it
example_rows = qa_ds.materialize().take(3)

In [5]:
for row in example_rows:
    print_wrapped("TEXT", row[DataSchema.ARTICLE])
    print_wrapped("QUESTIONS", row[DataSchema.MCQ_QUESTIONS])
    print_wrapped("ANSWERS", str(row[DataSchema.GROUND_TRUTH_MCQ_ANSWERS]))
    pprint.pprint("=" * 80, width=80)

TEXT:
By . Sean Poulter . PUBLISHED: . 20:04 EST, 10 March 2014 . | . UPDATED: . 04:33
EST, 11 March 2014 . Advances: The new system will make it easier to move money
around . Technology to allow direct payments between mobile phones was unveiled
by the big banks yesterday. The system cuts out the need to remember sort codes
and bank account details. Instead, you type on your phone the mobile number of
the person or business you want to pay. The ‘Paym’ transfers, which will be
password-protected, need your bank account to be linked to your own mobile
number. Users will simply tap in the number of the recipient on their phone to
authorise an electronic transfer from one account to another. The industry hopes
the system will replace cheques, which are expensive to transport and process.
At the same time it could provide a substitute for cash to make relatively small
payments to tradesmen, window cleaners or gardeners. The idea is that millions
of people will have their mobile phone numbe

### Part (b): Generate Summaries + Scores

Next, we will generate 10 summaries for each article in the training set and score them with our Q&A judging setup. 

The following command will run the [generate_summaries_and_scores.py](src/scripts/generate_summaries_and_scores.py) script, which takes in the folder with generated questions + articles and stores the results to a new folder of `.parquet` files. This script will use the model under training to produce 10 summaries per each example on all of the input data examples. Followed by each summarization, it will also perform summary accuracy measurement, asking the down-stream LLM to answer the questions generated earlier solely based on the summaries generated by the desired model. 

🔄 REPLACE the S3 URI in [`configs/summary_generation/8b_judge/mistral_finetuned_eval.yaml`](configs/summary_generation/8b_judge/mistral_finetuned_eval.yaml) with the path to the folder with generated questions from the previous job


This job will take ~320 min for 8B on 14 A10Gs (~ $76) and for 70B on A100s (~ $125) given the default configurations.


```bash
anyscale job submit -f configs/jobs/8b_judge/generate_summaries_train_job.yaml 
# Optional: use the 70b model for better performance (runs on A100s)
# anyscale job submit -f configs/jobs/70b_judge/generate_summaries_train_job.yaml
```

💡 INSIGHT: Feel free to modify the `concurrency` argument to increase throughput and reduce overall time taken for job. Note that for high values the job might not acquire the specified resources and this indicates a lack of availability of GPUs. Try decreasing the `concurrency` argument for reference model or the judge.  

🔄 REPLACE the below S3 URI with the link to the generated summaries from the job. You can optionally skip the previous with the example dataset below.

In [ ]:
# replace with the link to the generated summaries
summary_folder = "s3://air-example-data/preference-tuning-summarization-example/summary_generation_base/train/"
summary_ds = ray.data.read_parquet(summary_folder)
example_rows = summary_ds.take(1)

In [14]:
for row in example_rows:
    print_wrapped("TEXT", row[DataSchema.ARTICLE])
    print_wrapped("QUESTIONS", row[DataSchema.MCQ_QUESTIONS])
    print_wrapped("MODEL GENERATED SUMMARY", row[DataSchema.SUMMARY_GENERATION_RAW_OUTPUT])
    print_wrapped("ANSWERS", str(row[DataSchema.GROUND_TRUTH_MCQ_ANSWERS]))
    print_wrapped("JUDGE ANSWERS FROM SUMMARY", str(row[DataSchema.JUDGE_MCQ_ANSWERS]))
    pprint.pprint("=" * 100, width=80)

TEXT:
By . Kerry Mcqueeney . UPDATED: . 04:15 EST, 6 March 2012 . The wife of a
British man facing arms dealing charges in the United States has described a
judge's decision to remand him in custody ahead of his trial in the United
States as 'heartbreaking'. Elaine Tappin said it was an 'outrage' that her
65-year-old husband Christopher was refused bail after he was extradited to the
United States two weeks ago. Judge Robert Castaneda ruled Tappin must remain in
custody after US prosecutors told the federal court in El Paso, Texas, he may be
a 'danger to the community' if released. Accused: An artist's impression of
Christopher Tappin at his bail hearing in the El Paso Federal Courthouse .
Heartbroken: Elaine Tappin with her husband Christopher, before he was
extradited to the U.S. Mrs Tappin, 62, of Orpington, Kent, said: 'This is an
outrage. God only knows how he'll bear up. It's heartbreaking.' Tappin has spent
23 hours a day locked in his cell at the Otero County detention centre i

### Part (c): Generate Preference Tuning Data

The final step for getting our data ready! We'll now generate (chosen, rejected) summary pairs for each article based on the scores.

The following command will run the [generate_dpo_data.py](src/scripts/generate_dpo_data.py) script, which takes in the folder of summaries and outputs `.jsonl` files for training and validation.

🔄 REPLACE the S3 URI in [`configs/training_data_generation/mistral_8b.yaml`](configs/training_data_generation/mistral_8b.yaml) with the path to the folder with generated summaries from the previous job

Run the following command in the terminal to generate DPO data:
```bash
export PYTHONPATH=$PYTHONPATH:src
python src/scripts/generate_dpo_data.py configs/training_data_generation/mistral_8b.yaml
```

In [ ]:
# Inspect the results
# Replace with the link to your validation file
validation_file = "s3://air-example-data/preference-tuning-summarization-example/dpo_training_data/valid.jsonl"

valid_ds = ray.data.read_json(validation_file)
example_rows = valid_ds.take(1)

In [9]:
for row in example_rows:
    print_wrapped("PROMPT", row["chosen"][0]["content"])
    print_wrapped("CHOSEN RESPONSE", row["chosen"][1]["content"])
    print_wrapped("REJECTED RESPONSE", row["rejected"][1]["content"])

PROMPT:
Given the following text, create a very short summary that is at most 2
sentences.  Text: By . Tamara Cohen, Political Reporter . PUBLISHED: . 18:32
EST, 27 January 2013 . | . UPDATED: . 08:48 EST, 28 January 2013 . Deputy Prime
Minister Nick Clegg and his wife Miriam are determined to keep the education of
their 11-year-old son 'out of politics' Nick Clegg yesterday defended the
possibility he may send his children to private schools as it emerged he and his
wife Miriam have not even visited their local state school. He said the
education of his 11-year-old son Antonio, who starts secondary school this year,
should not be used as 'a political football' and that the couple would do
'what's best' for their children although he was braced for criticism. Last week
the Liberal Democrat leader told listeners to his radio show he would send his
son to a private school if he failed to find a place in a good comprehensive,
saying he would use the state system 'if it works out', but tha

# Step 2: Fine-tuning

Now that we have the pre-processed dataset, we are ready to fine-tune `Mistral-7B-Instruct-v0.1` using DPO. On Anyscale, we've created an easy-to-use interface to do preference-tuning using DPO. We leverage Ray to overlap reference model log-probability calculation with model training to improve GPU utilization. Most implementations compute log probabilities synchronously with model training,

![hf model](./assets/hf_dpo.png)

While our implementation using Ray is asynchronous:  


![assistant model](./assets/anyscale_dpo.png)

Further, our use of Ray Data also implies that the compute configuration for the reference model can be completely decoupled with the policy model. For example, reference model calculation can run on a different node (with configurable number of GPUs, etc) with zero code changes needed. 

> **NOTE** Make sure you've gove over the [user guides](https://docs.anyscale.com/category/fine-tuning-beta) for fine-tuning to understand the different configurations available

To get started with DPO training, we provide the config for DPO in [configs/mistral_dpo_summarization.yaml](configs/mistral_dpo_summarization.yaml) . You can add your `WANDB_API_KEY` as an environment variable in the dependencies tab if you wish to track progress of your run on WandB.


 🔄 REPLACE the training and validation file paths in the config with the output file paths in the previous step

In [10]:
!cat configs/mistral_dpo_summarization.yaml

model_id: mistralai/Mistral-7B-Instruct-v0.1
# Example summarization dataset with 10k examples for training with an average of 2.2k tokens per sample.
# Make sure to replace `train_path` and `valid_path` with the path to the files you generated
train_path: s3://air-example-data/preference-tuning-summarization/train.jsonl
valid_path: s3://air-example-data/preference-tuning-summarization/valid.jsonl

task: "preference_tuning"
context_length: 4096
# For DPO, it is recommended to set a high `num_data_blocks_per_device` to not bottleneck the logp processor.
num_data_blocks_per_device: 32
# Runs training on 12 GPUs
num_devices: 12
train_batch_size_per_device: 2
eval_batch_size_per_device: 2
learning_rate: 5e-6
num_epochs: 3
no_gradient_checkpoint: False
# Deepspeed configuration, you can provide your own deepspeed setup
deepspeed:
  config_path: configs/zero_3.json
worker_resources:
  accelerator_type:A10G: 1
padding: "longest"
preference_tuning_config:
  beta: 0.01
  logprob_processor_scali

You can fine-tune the model now by submitting it as an Anyscale job: 

```bash
anyscale job submit configs/jobs/mistral_dpo_job.yaml
```

This should take about 10 hours for the 8B model on 16 A10s (2 nodes with 8xA10), and about 7 hours for the 70B model on 8 A100s (1 node with 8xA100). 

💡 INSIGHT: This fine-tuning job inherits the compute configuration of the current workspace - meaning the job runs on a CPU-only head node with auto-scaling enabled. Sometimes, the nodes you get with auto-scaling can be in-efficient for fine-tuning due to inter-node communication costs (Say you get 2 4xA10 nodes instead of 8xA10s due to availability). You can explicitly set the compute configuration for the job with a set number of worker nodes to avoid this (but this might involve more wait times).
 - More on compute configurations here: https://docs.anyscale.com/configuration/compute-configuration 
 - The complete Anyscale Job API reference: https://docs.anyscale.com/reference/job-api#jobconfig 

# Step 3: Evaluation

Let's evaluate our trained model. Here we'll use two baselines: (1) the base model before finetuning (reference model in DPO) and (2) GPT-4o.

## Evaluation strategy

Our evaluation strategy involves the same Q&A scoring system as used while generating the preference data. 

<p align="center">
  <img src="./assets/eval.png?" alt="Evaluation" width=800>
</p>

We evaluate the baseline model and the trained DPO model on the test set. 

## Obtain summaries on the test set
First, we'll need to obtain the summaries (and scores) for both the models on the given test set. 

For the baseline model, you can simply run the below command:
```bash
anyscale job submit -f configs/jobs/8b_judge/generate_summaries_eval_baseline_job.yaml
# Optional: use the 70b model for better performance (runs on A100s)
# anyscale job submit -f configs/jobs/70b_judge/generate_summaries_eval_baseline_job.yaml 
```

This should take ~10 min for the 8B model on 8 A10s ( < $2) and the 70B model on A100s (< $4).

For the fine-tuned DPO model, we provide a dummy config in [configs/summary_generation/8b_judge/mistral_finetuned_eval.yaml](configs/summary_generation/8b_judge/mistral_finetuned_eval.yaml). If you used the default training config provided, the model would be trained using LoRA and you should have a path to the LoRA weights. 

In [11]:
!cat configs/summary_generation/8b_judge/mistral_finetuned_eval.yaml

mode: eval
input_folder: s3://air-example-data/preference-tuning-summarization-example/qa_generation/qa_annotations_full_test
inference_type: offline
model_inference_config:
  # Modify with s3 link to full param weights if you did full-param training
  model_id_or_path: mistralai/Mistral-7B-Instruct-v0.1

  # Add path to lora weights here. If you did full param training, you can instead remove this field.
  adapter_id_or_path: s3://large-dl-models-mirror/finetuning_template/mistral_dpo_summarization_lora

  temperature: 0
  top_p: 0.95
  scaling_config:
    batch_size: 64
    concurrency: 4
    num_gpus_per_instance: 1
    accelerator_type: A10G
num_generations: 1
judge_inference_config:
  model_id_or_path: meta-llama/Meta-Llama-3.1-8B-Instruct
  temperature: 0
  scaling_config:
    batch_size: 64
    concurrency: 3
    num_gpus_per_instance: 2
    accelerator_type: A10G
num_mcq_questions: 5


 🔄 REPLACE the `adapter_id_or_path` entry in the config with the path to your LoRA weights before proceeding (if you used the fine-tuning defaults). Alternatively, make sure to replace `model_id_or_path` entry (and remove the `adapter_id_or_path` entry) if you did full-param fine-tuning.

We are now ready to evaluate our fine-tuned model: 

```bash
anyscale job submit -f configs/jobs/8b_judge/generate_summaries_eval_finetuned_job.yaml
# Optional: use the 70b model for better performance (runs on A100s)
# anyscale job submit -f configs/jobs/70b_judge/generate_summaries_eval_finetuned_job.yaml
```

In the logs for the above jobs, you should see the final path to the output summaries for both the models. 

Optionally, you can also obtain the summaries and scores for the `gpt-4o` model from OpenAI. Simply run: 

```bash
anyscale job submit -f configs/jobs/8b_judge/generate_summaries_eval_gpt_job.yaml
# Optional: use the 70b model for better performance (runs on A100s)
# anyscale job submit -f configs/jobs/70b_judge/generate_summaries_eval_gpt_job.yaml
```

This should take about ~10 min for the 8B model on 8 A10s and the 70B model on 8 A100s. 

## Get Evaluation Statistics

We've provided a convenient script [get_eval_stats.py](src/scripts/get_eval_stats.py) to get evaluation statistics and obtain the "win rate" of the DPO model (the percentage of times the DPO model performs better than the baseline). We've provided an example configuration below. 

🔄 REPLACE the `--outputs-path` field and optionally the `--gpt4o-outputs-path` with the paths you generated from the above jobs.

```bash 
# make sure to substitute --outputs-path with your path
python src/scripts/get_eval_stats.py --outputs-path s3://air-example-data/preference-tuning-summarization-example/summary_generation_dpo_model/test/ --baseline-outputs-path s3://air-example-data/preference-tuning-summarization-example/summary_generation_base/test/  

# (Optional): if you obtained results for GPT-4o, you should uncomment and run the following command instead
# python src/scripts/get_eval_stats.py --outputs-path s3://air-example-data/preference-tuning-summarization-example/summary_generation_dpo_model/test/ --baseline-outputs-path s3://air-example-data/preference-tuning-summarization-example/summary_generation_base/test/  --gpt4o-outputs-path <add-path-to-gpt4o-results>
```

You should see the following results for the 70B model:

```text 
╒═════════════════════════════╤═══════════╤════════════╤═══════════╕
│           Metric            │   Model   │  Baseline  │  GPT-4o   │
╞═════════════════════════════╪═══════════╪════════════╪═══════════╡
│        Accuracy >=3         │ 65.4286 % │ 43.0476 %  │ 37.2381 % │
├─────────────────────────────┼───────────┼────────────┼───────────┤
│        Accuracy >=4         │ 25.7143 % │ 13.5238 %  │ 10.0000 % │
├─────────────────────────────┼───────────┼────────────┼───────────┤
│     Median Compression      │ 11.5794 % │ 12.7316 %  │ 8.0496 %  │
├─────────────────────────────┼───────────┼────────────┼───────────┤
│      Mean Compression       │ 13.0029 % │ 14.3444 %  │ 9.3554 %  │
├─────────────────────────────┼───────────┼────────────┼───────────┤
│      Summary Too Long       │ 0.0000 %  │  0.0000 %  │ 0.0000 %  │
├─────────────────────────────┼───────────┼────────────┼───────────┤
│ Contains Invalid Characters │ 0.0000 %  │  0.0952 %  │ 0.0000 %  │
╘═════════════════════════════╧═══════════╧════════════╧═══════════╛


Model Win Rate against Baseline: 74.0000 %
GPT-4o Win Rate against Baseline: 64.8095 %
```

Our fine-tuned model is able to generate much better summaries, that are more concise (compression ratio is lower) with lesser out-of-distribution characters (gibberish tokens) than the baseline. You can see more details on the same in our blog!

| **NOTE:** The evaluation results will differ if you used the 8B model which is less capable as a LLM-judge (and thus the numbers can be less accurate)

## Summary

Congrats! You have now fine-tuned an open source model on preference data. As a quick recap, here's what we demonstrated in this notebook:
1. Synthetically generating preference data for DPO 
2. DPO fine-tuning of a language model on the Anyscale Platform
4. Evaluating the model against the baseline and GPT-4o, and analysing the results.